In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
nb_classes = 10

In [ ]:
img = x_train[0]
print(img.shape)
plt.imshow(img, cmap="gray")

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
img = img.reshape(-1,28,28,1)
img = tf.image.convert_image_dtype(img, tf.float32)
W1 = tf.Variable(tf.random_normal([3,3,1,5], stddev=0.01))

In [ ]:
conv2d = tf.nn.conv2d(img, W1, strides=[1,2,2,1], padding="SAME")
print(conv2d)
sess.close()

In [ ]:
# sess = tf.InteractiveSession()
# sess.run(tf.global_variables_initializer())

In [ ]:
# conv2d_img = conv2d.eval()
# conv2d_img = np.swapaxes(conv2d, 0, 3)

In [ ]:
# for i, one_img in enumerate(conv2d_img):
#     plt.subplot(1,5,i+1), plt.imshow(one_img.reshape(14, 14), cmap="gray")

# CNN 실습 예제

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
nb_classes = 10

###### reshape를 통해서 vector로 변환

In [ ]:
input_shape = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(-1, input_shape) 
y_train = to_categorical(y_train, nb_classes)

###### array to tensorflow

In [ ]:
X_train = tf.image.convert_image_dtype(X_train, tf.float32)
y_train = tf.image.convert_image_dtype(y_train, tf.float32)

In [ ]:
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

In [ ]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding="SAME")
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides = [1,2,2,1], padding="SAME")

In [ ]:
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding="SAME")
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides = [1,2,2,1], padding="SAME")
L2 = tf.reshape(L2, [-1, 7 * 7 * 64])

In [ ]:
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10], initializer= tf.compat.v1.keras.initializers.glorot_normal)

In [ ]:
b = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b

In [ ]:
learning_rate = 0.1
training_epochs =15
batch_size = 100
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# epoch 돌리는 부분

In [ ]:
print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(60000 / batch_size)
    for i in tqdm(range(total_batch)):
        batch_xs, batch_ys = X_train[(i*batch_size):batch_size+(i*batch_size)].eval(), y_train[(i*batch_size):batch_size+(i*batch_size)].eval()
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y:batch_ys})
        avg_cost += c / total_batch
    print("Epoch:", "%04d" % (epoch + 1), "cost =", "{:.9f}".format(avg_cost))
print("Learning Finished!")

# 예측하기

In [ ]:
input_shape = X_test.shape[1] * X_test.shape[2]
X_test = X_test.reshape(-1, input_shape) 
y_test = to_categorical(y_train, nb_classes)

In [ ]:
X_test = tf.image.convert_image_dtype(X_test, tf.float32)
y_test = tf.image.convert_image_dtype(y_test, tf.float32)

In [ ]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))